In [18]:
# get data
from binance.spot import Spot 
from constants import price_cols, PERIOD, SYMBOL, DURATION_EACH_DAY, DATA_POINT_ONE_DAY, NUM_OF_DAYS

# train data
import pandas as pd
from datetime import datetime

# enviroment
import os
from dotenv import load_dotenv
load_dotenv("../../env/app.env")

True

In [13]:
BINANCE_API_KEY = os.environ.get("BINANCE_API_KEY")
BINANCE_API_SECRET = os.environ.get("BINANCE_API_SECRET")
client = Spot(key=BINANCE_API_KEY, secret=BINANCE_API_SECRET)

## Read old prices

In [30]:
price = pd.read_csv(f"../../datastore/price/{SYMBOL}_{PERIOD}.csv")
price

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1517500800000,9326.23,9340.00,9195.89,9217.00,487.476964,1517501699999,4.506285e+06,4546,171.643712,1.586177e+06
1,1517501700000,9208.09,9275.00,9130.00,9145.00,652.169234,1517502599999,6.004393e+06,6930,334.699485,3.083885e+06
2,1517502600000,9144.98,9257.40,9020.00,9184.99,1028.146331,1517503499999,9.361539e+06,8632,376.048437,3.429102e+06
3,1517503500000,9184.98,9270.00,9011.51,9066.66,770.943401,1517504399999,7.037635e+06,6722,356.335133,3.256645e+06
4,1517504400000,9058.78,9156.98,9040.00,9106.22,490.060336,1517505299999,4.458077e+06,4557,244.406918,2.225572e+06
...,...,...,...,...,...,...,...,...,...,...,...
174058,1673004600000,16746.93,16750.99,16727.63,16747.39,2220.155130,1673005499999,3.716158e+07,64789,1048.957450,1.755804e+07
174059,1673005500000,16747.39,16755.00,16736.98,16738.22,1629.538980,1673006399999,2.728818e+07,46912,753.431990,1.261708e+07
174060,1673006400000,16738.74,16749.00,16730.12,16737.15,1763.072670,1673007299999,2.950915e+07,50505,863.799650,1.445800e+07
174061,1673007300000,16737.15,16738.28,16719.09,16721.17,1919.983110,1673008199999,3.212120e+07,52028,945.981120,1.582643e+07


## Fetch new prices

In [31]:
start_timestamp = price.iloc[-1, 0]
today = datetime.utcnow().replace(hour=23, minute=0, second=0, microsecond=0)
today_timestamp = int(today.timestamp() * 1000)

In [32]:
for timestamp in range(start_timestamp, today_timestamp, DURATION_EACH_DAY):
    data = client.klines(SYMBOL, PERIOD, limit=1000, startTime=timestamp, endTime=timestamp + DURATION_EACH_DAY)
    if start_timestamp == timestamp:
        new_price_df = pd.DataFrame(data, columns=price_cols)
    else:
        new_price_df = pd.concat([new_price_df, pd.DataFrame(data, columns=price_cols)], axis=0)

new_price_df.drop("Unused field, ignore", axis=1, inplace=True)

## Concatenate old and new prices and save

In [34]:
price = pd.concat([price, new_price_df])
price.drop_duplicates("Kline open time", inplace=True)
price.to_csv(f"../../datastore/price/{SYMBOL}_{PERIOD}.csv", index=False)